In [21]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [22]:
import os

# Set environment variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'cortex'

# Get keys from the environment
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if langchain_api_key:
    os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
else:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment.")

if groq_api_key:
    os.environ['GROQ_API_KEY'] = groq_api_key
else:
    raise ValueError("GROQ_API_KEY is not set in the environment.")

PART 1 - OVERVIEW   

In [23]:
import os
from dotenv import load_dotenv, find_dotenv
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

# Load environment variables from .env file
load_dotenv(find_dotenv())

# Disable LangSmith tracing to prevent API errors
os.environ["LANGCHAIN_TRACING_V2"] = "false"

#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Initialize embeddings model
model_name = "BAAI/bge-small-en"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs={"normalize_embeddings": True}
)

# Create vector store
vectorstore = FAISS.from_documents(documents=splits, embedding=hf_embeddings)
retriever = vectorstore.as_retriever()

#### RETRIEVAL AND GENERATION ####

# Define RAG prompt
prompt = hub.pull("rlm/rag-prompt")

# Initialize Groq LLM (Llama3-8b)
llm = ChatGroq(model="llama3-8b-8192", temperature=0)

# Format documents for context
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Query the system
question = "What is Task Decomposition?"
print(rag_chain.invoke(question))

Task Decomposition is the process of breaking down a complex task into smaller, manageable subtasks or steps. This can be done using various methods, such as simple prompting, task-specific instructions, or human inputs. The goal is to create a clear plan or outline for achieving the desired outcome.


PART 2 - INDEXING

In [24]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

Document Loaders

In [25]:
# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

Splitter

In [26]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

Text Embedding Models

In [41]:
from langchain_huggingface import HuggingFaceEmbeddings

# Define model and encoding configurations
model_name = "BAAI/bge-small-en"
device = "cpu"
encode_kwargs = {"normalize_embeddings": True}

# Initialize the HuggingFaceBgeEmbeddings without 'model_kwargs'
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, encode_kwargs=encode_kwargs
)

# Embed the query and document
query_result = hf_embeddings.embed_query(question)
document_result = hf_embeddings.embed_query(document)

# Print the length of the query embedding
print(f"Query embedding length: {len(query_result)}")

Query embedding length: 384


In [42]:
import numpy as np

# Cosine similarity function
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# Calculate cosine similarity
similarity = cosine_similarity(query_result, document_result)

# Print result
print("Cosine Similarity:", similarity)


Cosine Similarity: 0.8786946957859941


Vectorstores

In [43]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [44]:
from langchain_community.vectorstores import FAISS

# Create a FAISS vector store
vectorstore = FAISS.from_documents(documents=splits, embedding=hf_embeddings)

# Create a retriever from the FAISS vector store
retriever = vectorstore.as_retriever()

# Now you can use the retriever to search documents or queries
print("FAISS retriever created successfully.")

FAISS retriever created successfully.


PART 3 - RETRIEVAL

In [45]:
docs = retriever.invoke("What is Task Decomposition?")

In [46]:
len(docs)

4

In [47]:
docs

[Document(id='5e3060de-7eb3-4c6d-a422-d19c3119d08a', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='The AI assistant can parse user input to several tasks: [{"task": task, "id", task_id, "dep": dependency_task_ids, "args": {"text": text, "image": URL, "audio": URL, "video": URL}}]. The "dep" field denotes the id of the previous task which generates a new resource that the current task relies on. A special tag "-task_id" refers to the generated text image, audio and video in the dependency task with id as task_id. The task MUST be selected from the following options: {{ Available Task List }}. There is a logical relationship between tasks, please note their order. If the user input can\'t be parsed, you need to reply empty JSON. Here are several cases for your reference: {{ Demonstrations }}. The chat history is recorded as {{ Chat History }}. From this chat history, you can find the path of the user-mentioned resources for your task planning.

PART 4 - GENERATION

In [48]:
# Import required modules
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

# Define the prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# Create the prompt using LangChain's ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)

prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [49]:
# Example of using ChatGroq 
llm = ChatGroq(model="llama3-8b-8192", temperature=0)

In [50]:
# Chain
chain = prompt | llm

In [51]:
# Run
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

AIMessage(content='According to the context, Task Decomposition is a process in which a complicated task is broken down into smaller and simpler steps. This is achieved through techniques such as Chain of Thought (CoT) and Tree of Thoughts (Yao et al. 2023), which involve decomposing the problem into multiple thought steps and generating multiple thoughts per step, creating a tree structure. Task decomposition can be done by LLM with simple prompting, task-specific instructions, or human inputs.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 917, 'total_tokens': 1013, 'completion_time': 0.08, 'prompt_time': 0.122967625, 'queue_time': 0.23796503700000002, 'total_time': 0.202967625}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-e8bb9791-70e9-40d1-a07f-c9610ccbb6e4-0', usage_metadata={'input_tokens': 917, 'output_tokens': 96, 'total_tokens': 1013})

RAG Chains

In [52]:
from langchain import hub

# Pull prompt from LangChain Hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [53]:
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [54]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Define the pipeline
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # Use context and question
    | prompt_hub_rag  # Prompt for RAG
    | llm  # The LLM model for generating the response
    | StrOutputParser()  # Parse the output to string
    | (lambda x: "\n".join([q.strip() for q in x.split("\n") if q.strip()])) 
)

# Invoke the chain with a user question
result = rag_chain.invoke("What is Task Decomposition?")

# Print the wrapped result
print(result)

Task Decomposition is a process that breaks down a complicated task into smaller and simpler steps, allowing an agent to plan ahead and utilize more test-time computation. This process can be done through various methods, including using LLM with simple prompting, task-specific instructions, or human inputs.
